# Prediction
Sean Wade

 - logistic regression l1 better

In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, VotingClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample

In [65]:
train = pd.read_csv('clean_data/train_final.csv', index_col=0)
test = pd.read_csv('clean_data/test_final.csv', index_col=0)

In [42]:
# get all the columns by type
numeric_cols = list(set(test.columns.tolist()).difference(set(one_hot_cols)))
one_hot_cols = test.filter(regex=("^ONEHOT*")).columns.tolist()
bool_cols = test.select_dtypes('bool').columns.tolist()

In [7]:
#train = train.select_dtypes(['int', 'float'])

In [28]:
y = train.TARGET
X = train.drop(['TARGET'], axis=1)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [35]:
X_train, y_train = balance_dataset(X_train, y_train)

In [54]:
rf = RandomForestRegressor()
lr = LogisticRegression(penalty='l1')

In [61]:
model = VotingClassifier(
    estimators= [
        ('logistic regression', lr),
        ('random forrest', rf)
    ],
    voting='hard',
    weights=[1,1]
)

In [62]:
model.fit(X_train[numeric_cols + one_hot_cols], y_train)

VotingClassifier(estimators=[('logistic regression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)...timators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [63]:
y_hat = model.predict(X_train[numeric_cols + one_hot_cols])

TypeError: Cannot cast array data from dtype('float64') to dtype('int64') according to the rule 'safe'

In [ ]:
roc_auc_score(y_train, y_hat)

In [60]:
scores = cross_val_score(model, X[numeric_cols + one_hot_cols], y, cv=5, scoring='roc_auc')
scores.mean()

AttributeError: predict_proba is not available when voting='hard'

## Final Prediction

In [ ]:
test_ids = test.SK_ID_CURR
test.drop('SK_ID_CURR', axis=1, inplace=True)
test = test.select_dtypes(['int', 'float'])

In [ ]:
y_hat = model.predict(test)

In [ ]:
ans = pd.DataFrame(y_hat, index=test_ids, columns=['TARGET'])

In [ ]:
ans.to_csv('prediction.csv')

In [34]:
def balance_dataset(X_train, y_train):
    # Seperate unbalanced classes
    df_majority = X_train[y_train == 0]
    df_minority = X_train[y_train == 1]
    y_majority = y_train[y_train == 0]
    y_minority = y_train[y_train == 1]

    # Upsample minority class
    df_minority_up, y_minority_up = resample(df_minority, y_minority, 
                                     replace=True,               # sample with replacement
                                     n_samples=len(df_majority), # to match majority class
                                     random_state=123)

    # Combine majority class with upsampled minority class
    df_upsampled = pd.concat([df_majority, df_minority_up])
    y_upsample = pd.concat([y_majority, y_minority_up])

    return df_upsampled, y_upsample